# OpenAI Responses API: Advanced Tweet Analysis with File & Web Search Integration

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Connecting to a MongoDB database** to store and retrieve tweets.
- **Extracting tweets** and converting them into a structured format for further analysis.
- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval 
and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API and interact with a MongoDB database, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API
- **`pymongo`**: A Python driver for MongoDB to store and retrieve tweets.

In [1]:
pip install openai pymongo -q

Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [2]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Connect to MongoDB

In [4]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
# tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

## Extract Tweets from MongoDB

In [5]:
filter={

    
}
project={
    'tweet.text': 1,
    '_id':0
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

After retrieving tweets from MongoDB, we convert the query result into a list format for easier processing.
The data is then serialized into a JSON-formatted string, ensuring it can be properly stored and shared across different services.
Using `io.BytesIO`, we create an in-memory JSON file, eliminating the need for disk writes.
This approach is particularly useful for applications that require temporary file storage, such as uploading datasets
to OpenAI's file search API or cloud storage for further analysis.

In [6]:
result_list = list(result)

# Convert result list to JSON string
json_data = json.dumps(result_list, default=str, indent=4)

# Create an in-memory JSON file
json_bytes = io.BytesIO(json_data.encode("utf-8"))
json_bytes.name = "tweet.json" 

In [7]:
print('Number of tweets: ',len(result_list))

Number of tweets:  69


## Initialize OpenAI Client

In [8]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [9]:
vector_store = client.vector_stores.create(
    name="tweet_base"
)
vector_store_id = vector_store.id
# print(vector_store_id)

### Upload Tweets File

In [10]:
file = client.files.create(
            file=json_bytes,
            purpose="assistants",)

file_id = file.id
# print(file_id)

### Attach File to Vector Store

In [11]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

# print(attach_status.id)

### Query the Vector Store

In [14]:
query = "jmu sports"

In [15]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

\n\n\u26be\ufe0f Series win at home over ODU \n\ud83e\udd4e Series sweep at JMU \n\ud83c\udfbe Ran w
 Relevant score: 0.6150618121732849
Here's what HC Matt Deggs had to say in part of his opening statement, recapping last week's games\n
 Relevant score: 0.561936313614552
\n\n\u26be\ufe0f Series win at home over ODU \n\ud83e\udd4e Series sweep at JMU \n\ud83c\udfbe Ran w
 Relevant score: 0.5286285434213078
St Clair Academy\n\u23f07 p.m.\n\ud83d\udcfa https://t.co/mAYLOFicKB\nWVU vs. JMU\n\u23f08:30 p.m.\n
 Relevant score: 0.518714128495876
{
        "tweet": {
            "text": "RT @HNlEHupY4Nr6hRM: JMU\u78ef\u5b50\u306b\u3066\uff92\uff
 Relevant score: 0.5165195458111294


## OpenAI Response API

### Simple Response

In [16]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [17]:
display(Markdown(simple_response.output_text))

James Madison University (JMU) has a vibrant sports program known for its strong presence in various collegiate athletics. The JMU Dukes compete in the NCAA Division I and are part of the Sun Belt Conference for most sports. Here's an overview of some key sports:

1. **Football**: JMU has a strong football tradition and gained national attention for winning the FCS National Championship in 2004 and 2016. The Dukes play their home games at Bridgeforth Stadium.

2. **Basketball**: Both the men's and women's basketball teams are competitive, playing their home games at the JMU Convocation Center. The women's team, in particular, has been successful in conference play and has made several NCAA tournament appearances.

3. **Soccer**: JMU has competitive men's and women's soccer programs, with the women’s team often contending for top spots in their conference.

4. **Lacrosse**: The women's lacrosse team is particularly strong, having won the NCAA National Championship in 2018.

5. **Baseball and Softball**: Both teams have competitive programs, with the softball team reaching the Women's College World Series in 2021, bringing national prominence to the program.

JMU offers a variety of other sports as well, including track and field, volleyball, swimming and diving, and more, with a strong emphasis on both athletic excellence and academic achievement.

### File Search Response

In [18]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [22]:
display(Markdown(file_search_response.output_text))


JMU sports have been active recently with several events and achievements:

1. JMU had a series sweep against ODU, extending their win streak to five games.
2. The JMU Rocket League teams participated in the NACE playoffs.
3. JMU's lacrosse team is ranked 13th in the latest IWLCA poll.
4. Paul Lewis, brother of JMU legend Matt Lewis, has entered the transfer portal.

These highlights reflect the ongoing activities and achievements of JMU's sports teams.

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [23]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [24]:
display(Markdown(web_search_response.output_text))

James Madison University's (JMU) athletic programs have experienced significant milestones and achievements recently.

**Football Achievements:**

- **Historic Bowl Appearance:** In the 2023 season, JMU's football team achieved an 11–1 record, leading the Sun Belt Conference's East Division. Despite initial ineligibility due to NCAA transition rules, the Dukes were granted a bowl berth due to a shortage of eligible teams. They participated in the Armed Forces Bowl on December 23, 2023, marking the program's first-ever bowl game. ([en.wikipedia.org](https://en.wikipedia.org/wiki/2023_Armed_Forces_Bowl?utm_source=openai))

- **NCAA Waiver Denial:** Earlier in the season, JMU sought a waiver from the NCAA to expedite their transition from the Football Championship Subdivision (FCS) to the Football Bowl Subdivision (FBS), aiming to become bowl-eligible sooner. However, the NCAA denied this request, maintaining the standard two-year transition period. ([abcnews.go.com](https://abcnews.go.com/Sports/james-madison-2-denied-football-postseason-waiver/story?id=104931737&utm_source=openai))

- **Upcoming Matchups:** Looking ahead, JMU has scheduled a game against Louisville on September 6, 2025. Additionally, their previously planned game at Virginia Tech has been rescheduled to September 26, 2026. ([jmusports.com](https://jmusports.com/news/2023/10/23/football-jmu-to-play-at-louisville-in-2025-virginia-tech-moved-to-2026.aspx?utm_source=openai))

**Other Sports Highlights:**

- **Men's Basketball:** The men's basketball team made headlines by entering the AP Top 25 for the first time in program history. They began the season with a notable victory over then-No. 4 Michigan State, winning 79-76 in overtime. ([wtvr.com](https://www.wtvr.com/sports/james-madison-riding-13-game-winning-streak-november-15-2023?utm_source=openai))

- **Women's Basketball and Volleyball:** JMU's women's basketball and volleyball teams both secured Sun Belt Conference championships in the 2022-2023 season, showcasing the university's broad athletic excellence. ([wtvr.com](https://www.wtvr.com/sports/james-madison-riding-13-game-winning-streak-november-15-2023?utm_source=openai))

These accomplishments underscore JMU's growing prominence in collegiate athletics across multiple sports. 

### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [25]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

James Madison University's (JMU) athletic programs have experienced significant milestones and achie

### Continue Query with Web Search

In [26]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [27]:
display(Markdown(continue_search_response.output_text))

Certainly! Here are some recent developments in James Madison University (JMU) athletics:

**Football Program Updates:**

- **Head Coach Transition:** In November 2023, JMU's head football coach, Curt Cignetti, accepted the head coaching position at Indiana University. Cignetti led the Dukes to significant success during his tenure, including an 11–1 record in the 2023 season. ([si.com](https://www.si.com/college/group-five/tag/james-madison?utm_source=openai))

- **Transfer Portal Activity:** The football program has been active in the transfer portal:
  - In January 2024, former Washington quarterback Dylan Morris committed to JMU, bringing experience from the Pac-12 Conference. ([si.com](https://www.si.com/college/group-five/tag/james-madison?utm_source=openai))
  - Additionally, Vanderbilt tight end Logan Kyle transferred to JMU, adding depth to the offensive lineup. ([si.com](https://www.si.com/college/group-five/tag/james-madison?utm_source=openai))

**Men's Basketball Achievements:**

- **National Ranking:** The men's basketball team achieved a historic milestone by entering the AP Top 25 rankings for the first time in program history. As of December 2023, they were ranked No. 20, reflecting their strong performance throughout the season. ([deadspin.com](https://deadspin.com/no-20-james-madison-learning-to-maintain-effort-faces-1851106882/?utm_source=openai))

- **Notable Victory:** The Dukes secured a significant win against Hampton University on December 16, 2023, overcoming a 10-point deficit to claim an 88-71 victory. This game highlighted the team's resilience and ability to perform under pressure. ([deadspin.com](https://deadspin.com/no-20-james-madison-learning-to-maintain-effort-faces-1851106882/?utm_source=openai))

These developments underscore the dynamic and evolving nature of JMU's athletic programs across multiple sports. 

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [28]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [29]:
display(Markdown(combined_search_response.output_text))

JMU sports have been active recently with several events and achievements:

1. **Baseball**: JMU had a series sweep against ODU and extended their win streak to five games.
   
2. **Esports**: JMU's Rocket League team participated in the NACE playoffs, facing off against WVU.

3. **Lacrosse**: JMU's lacrosse team is ranked 13th in the latest IWLCA poll.

4. **Football**: There have been several visits and recruitment activities, with players entering the transfer portal.

These highlights reflect the ongoing competitive spirit and achievements of JMU's sports teams.